## Atividade: Filtros Lineares e Integração de Sinais

Atividade sobre sistemas lineares como filtros, com aplicações em neurociência. Material baseado em notas de G. Boynton.




## A membrana como um filtro

Nas atividades anteriores, estudamos a dinâmica da membrana neuronal, modelada como um circuito RC. Utilizamos métodos geométricos (análise por "setinhas") e numéricos (como o Método de Euler) para determinar a resposta do sistema a pulsos de corrente, bem como os efeitos da alteração de valores de parâmetros.

Uma maneira alternativa de estudar este tipo de sistema (muito utilizada pelos engenheiros) consiste em interpretá-lo como um _filtro linear_. Para nossos propósitos, um filtro é uma transformação que recebe uma série temporal como entrada, e retorna outra série com o mesmo número de elementos.

Filtros têm muitas aplicações em neurociência, desde modelos para neurônios e sinapses, passando pela resposta hemodinâmica − importante para estudos de ressonância magnética funcional (fMRI), até modelos fisiológicos e comportamentais para processos de adaptação (por exemplo, adaptação ao contraste em estímulos visuais).  

## Integrando a equação da membrana linear

Vamos relembrar o nosso programa para integrar a equação da membrana linear (também chamada de _leaky integrator_, ou "integrador com vazamento"), com uma entrada (corrente injetada) $I(t)$:

$$
\frac{\mathrm{d}V}{\mathrm{d}t} = -\frac{V-V_{\mathrm{rep}}}{\tau} + \frac{I(t)}{C}
$$

Para este exercício, utilizaremos variáveis adimensionais, além de tomarmos o potencial de repouso da membrana $E_\mathrm{rep}=0$ e a capacitância $C=1$ (estas escolhas simplificam a equação e os programas mas não mudam em nada as conclusões). Estudaremos os efeitos de diferentes valores da constante temporal $\tau=RC$ sobre o sinal de entrada $I(t)$.

Primeiramente, investigaremos a relação entre a constante temporal $\tau$ e a ideia de "vazamento".

In [ ]:
# Integrando a equação da membrana pelo Método de Euler
from numpy import zeros, arange

def resposta_membrana(estimulo, tau=1, passos=5000, dt=0.001):
    t = arange(passos) * dt
    V = zeros(passos)
    for i in range(passos - 1):
        dVdt = estimulo[i] - V[i]/tau 
        V[i + 1] = V[i] + dt * dVdt
    return t, V

0) Utilize a função acima (ou implemente o Método de Euler caso não esteja usando _Python_) para integrar a dinâmica da membrana passiva (que dá 5000 passos de tamanho 0.001, num total de 5 unidades de tempo). Utilize o seguinte estímulo $I(t)$ (lembre-se de que $C=1$):

$$
  I(t) =
\begin{cases}
1,  & 0\leq t<1 \\
0, & t \geq 1
\end{cases}
$$

Faça um gráfico das séries temporais $V(t)$ e $I(t)$, para $\tau=10^6$ ("infinito" para nossos propósitos), $\tau=1$, $\tau=0.2$. Discuta os resultados obtidos em termos da ideia de "vazamento". Dica: a equação da membrana acima é idêntica à equação que rege o volume d'água num balde, com um furo de tamanho inversamente proporcional $\tau$, no qual é despejada água a uma taxa I(t)/C.


In [ ]:
from numpy import linspace, array
import matplotlib.pyplot as plt
t, V = resposta_membrana(zeros(5000))

I = zeros(5000)
I[t < 1] = 1

for tau in [1e6, 1, 0.2]:
    t, V = resposta_membrana(I, tau)
    plt.plot(t, V, label=f'τ={tau:g}')
plt.legend()

Considere o caso com $\tau=0.2$. Por que o potencial de membrana em $t\approx 1$ é $V\approx 0.2$? Tente relacionar estes resultados a com solução analítica para a equação do circuito RC discutida em aula e na primeira atividade de avaliação.

## Respostas a pulsos curtos

Os exemplos anteriores tomaram um estímulo de intensidade unitária (dimensão de potencial/tempo) e duração de uma unidade de tempo, cujo efeito total é aumentar em uma unidade o potencial de membrana. Isto pode ser visto facilmente no caso sem vazamento, para $\tau\gg 1$ acima. Na analogia com o balde, teríamos algo como uma vazão de um litro por segundo, durante um segundo, aumentando o volume d'água em um litro.

1a) O que acontence se tomarmos um estímulo de duração muito mais curta, por exemplo, 0.01 unidade de tempo, mas mantendo a mesma transferência de potencial (colocar o mesmo litro d'água no balde, mas em 0.01 s ao invés de em 1 s?) Escolha $\tau=0.2$ e faça um gráfico para ajudar na resposta.

1b) Repita o item acima, mas para um estímulo de duração $0.001$ unidade de tempo (mantendo o efeito total de tranferência de uma unidade de potencial). O resultado é muito diferente do item anterior?

In [ ]:
def pulso_quadrado(t, duracao, t0=0, area=1):
    I = zeros(len(t))
    I[(t >= t0) & (t < t0 + duracao)] = area / duracao
    return I

def resposta_degrau(duracao, pot_total=1.0, tau=0.2, t0=0, label=None):
    if label is None:
        label = round(duracao, 4)
    t, V = resposta_membrana(zeros(5000))
    I = pulso_quadrado(t, duracao, t0, pot_total)
    t, V = resposta_membrana(I, tau)
    plt.plot(t, V, label=label)
    return max(V)


plt.figure(2)
for duracao in [0.001, 0.002, 0.01]:
    resposta_degrau(duracao)
plt.legend()

As resposta são quase idênticas porque, para estímulos de duração muito menor do que a constante temporal do sistema (no caso, a membrana, ou o balde), não há "vazamento" significativo durante a aplicação do estímulo. Como estamos mantendo a mesma transferência de carga elétrica (ou total de água transferida), o efeito é praticamente o mesmo.

1c) Investigue a partir de qual duração do estímulo começamos a ver um efeito significativo na resposta, e como isto depende da constante temporal da membrana.

In [ ]:
fatores = [0.5, 1, 5, 10, 25, 50]

for tau in [0.2, 1]:
    plt.figure()
    plt.title(f'τ = {tau}')
    for pc in fatores:
        est = pc * tau / 100
        m = resposta_degrau(est, tau=tau, label=f'{pc}% τ')
    plt.legend()


O efeito que você acaba de observar é uma explicação para a [Lei de Bloch](https://journals.sagepub.com/doi/full/10.1177/2041669515593043), que descreve o fato de lampejos (muito curtos) de luz de durações distintas serem igualmente detectáveis, na medida em que carreguem a mesma quantidade de luz (é possível compensar um estímulo mais curto aumentando sua intensidade, para um mesmo efeito final). De fato, é comum modelar propriedades temporais das primeiros estágios do sistema visual como integradores com vazamento.

Pessoas familiares com fotografia provavelmente já se depararam com um fenômeno similar, que se manifesta no jogo entre tempo total de exposição (velocidade do obturador) e a abertura do diafragma da câmera.

Esta resposta estereotipada a estímulos "ráídos" que carregam uma unidade da variável em questão (potencial elétrico ou volume d'água nos nossos exemplos) chama-se _resposta ao impulso unitário_, e é extremamente importante na área de análise de sinais.

## Escala ou Homogeneidade

Deve estar claro, dada a maneira como o estímulo é aplicado, que ao dobrarmos o estímulo dobramos também o pico da resposta. Como a velocidade de retorno ao equilíbrio é proporcional ao valor da variável de estado ("tamanho das flechinhas" expresso por $V'(t)\propto -V$), note que a resposta inteira escala com o tamanho da entrada. 

2) Determine a resposta da membrana ($\tau=0.2$) a dois pulsos de duração 0.01 unidades de tempo, o primeiro começando em $t_1=0$, com intensidade $I_1=300$ e segundo em $t_2=3$, com intensidade $I_2=100$.

In [ ]:
def estimulo_resposta(t, V, I):
    plt.subplot(211)
    plt.plot(t, I)
    plt.ylabel('Estímulo')

    plt.subplot(212)
    plt.plot(t, V)
    plt.xlabel('Tempo')
    plt.ylabel('Potencial')


plt.figure(5)
t = arange(5000) * 0.001
duracao = 0.01
I = pulso_quadrado(t, duracao=duracao, area=3) +\
    pulso_quadrado(t, duracao=duracao, t0=3, area=1)
t, V = resposta_membrana(I, 0.2)
estimulo_resposta(t, V, I)

Note que o formato de cada trecho da resposta é idêntico, a menos de um fator de escala. Matematicamente, se a resposta de um sistema a um sinal $s(t)$ é $L(s(t))$, então para $c$ constante $L(cs(t))=cL(s(t))$. Tal propriedade é denominada _homogeneidade_. Relacione isso com o que você aprendeu em Geometria Analítica (ou Álgebra Linear) sobre vetores e multiplicação por escalares.

## Aditividade

No exercício anterior, o segundo pulso foi aplicado algum tempo depois do primeiro. O que acontece se os pulsos estiverem mais próximos no tempo?

3) Considere dois pulsos unitários de duração $0.01$, começando em $t_1=0$, $t_2=0.1$. Faça um gráfico sobrepondo as respostas da membrana ($\tau=0.2$) ao primeiro pulso, ao segundo pulsos e à soma dos dois. Descreva a resposta obtida.

In [ ]:
plt.figure(6, figsize=(6,2))

t, V = resposta_membrana(zeros(2000), passos=2000)
duracao = 0.01
I1 = pulso_quadrado(t, duracao=duracao)
I2 = pulso_quadrado(t, duracao=duracao, t0=0.1)

plt.plot(*resposta_membrana(I1, tau=0.2, passos=2000))
plt.plot(*resposta_membrana(I2, tau=0.2, passos=2000))
plt.plot(*resposta_membrana(I1+I2, tau=0.2, passos=2000), 'k', lw=2)
plt.ylim(0,2)

Note que a resposta à soma dos estímulos é a soma das respostas aos estímulos individuais:
$$
L(s_1+s_2) = L(s_1) + L(s_2)
$$

Esta propriedade é chamada de _aditividade_. Além disto, estamos assumindo que a constante temporal $\tau$ é fixa, fazendo com que a forma da resposta não dependa de quando ocorre o estímulo. Esta segunda propriedade é chamada de _invariância temporal_.



## Sistemas lineares

Um sistema que apresente as propriedades de homogeneidade e aditividade obedece ao _princípio da superposição_ (resposta devida a uma soma de entradas é igual à soma das respostas à cada entrada individualmente). 

Sistemas que obedecem ao princípio da superposição são ditos _Sistemas Lineares_, uma área com inúmeras aplicações (curso obrigatório para quase todas as Engenharias!). 

## Solução analítica para resposta ao impulso

Em uma atividade anterior, determinamos que a solução analítica para a equação da membrana (sem estímulo externo) era um decaimento exponencial para o potencial de repouso:

$$
V-V_\mathrm{rep}=V_0e^{-\frac{t}{\tau}}
$$

Notamos, das simulações anteriores, que o efeito de um pulso de curta duração depende somente da carga total $q$ transferida à membrana (área sob o gráfico do pulso $I(t)$), causando um salto praticamente instantâneo no potencial de membrana de $q/C$. Se tomarmos este valor (em relação ao potencial de repouso) como condição inicial, obteremos então uma expressão exata para a _resposta ao impulso para a membrana passiva_ (estudantes de Física devem reconhecer aqui algo parecido com uma Função de Green):

$$
V-V_\mathrm{rep} = \frac{q}{C}\exp{\left(-\frac{t}{\tau}\right)} 
$$


4) Compare a solução exata acima com o resultado de uma simulação para um pulso curto, começando em $t=0$, com $q/C=1$. 

In [ ]:
from numpy import exp 

plt.figure(7)
tau = 0.2

def resposta_impulso(t, tau):
    return exp(-t/tau)

t = arange(5000)*0.001
plt.plot(t, resposta_impulso(t, tau), '.')

resposta_degrau(0.005, tau=tau)

plt.xlim(0,1)

## Convolução

O princípio da superposição tem uma consequência importantíssima: como podemos visualizar _qualquer_ entrada complicada como uma sequência de pulsos deslocados e escalonados, então a resposta do sistema a tal entrada será dada por uma soma de respostas ao impulso unitário.

Comecemos com um exemplo, calculando (assim como fizemos acima) a resposta da membrana a três pulsos unitários muito curtos, aplicados nos passos $t_1, t_2, t_3$:

In [ ]:
passo = 0.01
t = arange(100) * passo  # tempo 

# passos em que ocorrem pulsos 
t1 = 0
t2 = 5
t3 = 10

tres_pulsos = zeros(len(t))  
tres_pulsos[[t1, t2, t3]] = 1 / passo # pulsos unitários

t, resp_3p = resposta_membrana(tres_pulsos, tau=0.2, passos=100, dt=passo)
plt.figure(8)
estimulo_resposta(t, resp_3p, tres_pulsos)

A resposta (potencial de membrana) para um dado instante de tempo pode ser prevista pela soma da resposta aos três pulsos. Cada uma das entradas gera uma resposta com o mesmo formato.

Consideremos a figura obtida acima. Sobre ela, faremos o gráfico da resposta individual a cada um dos 3 pulsos (serão três curvas a mais, com a mesma forma mas deslocadas no tempo). 

In [ ]:
# resposta a impulso: apenas um pulso unitário em t=0
pulso_em_zero = zeros(len(t))
pulso_em_zero[0] = 1/passo
t, resp_impulso = resposta_membrana(pulso_em_zero, tau=0.2,
                                  passos=100, dt=passo)

# três respostas a pulso individual, deslocadas no tempo
for passo_pulso in [t1, t2, t3]:
    t_pulso = passo_pulso*passo
    plt.plot(t + t_pulso, resp_impulso,'-g')

Como vimos, a resposta em um instante arbitrário de tempo, por exemplo no passo 15, é a soma destas três respostas ao impulso unitário, cada uma calculada em um instante diferente. Vejamos o valor de cada uma delas (curvas verdes) no 15º passo (aparecerão como estrelas no gráfico acima), e a soma destes 3 valores (aparecerá como quadrado azul acima).

In [ ]:
pt = 15
for t_pulso in [t1, t2, t3]:
    plt.plot(t[pt], resp_impulso[pt-t_pulso], '*g')
    
resposta_15 = resp_impulso[pt-t1] +\
                resp_impulso[pt-t2] +\
                resp_impulso[pt-t3]
plt.plot(t[pt], resposta_15, 'sb')

Uma outra maneira de ver o resultado acima é: para _qualquer_ estímulo, a resposta no passo 15 será a soma de "cópias" da função resposta ao estímulo, deslocadas no tempo e escaladas pelo estímulo (devemos chegar ao mesmo resultado da figura anterior):

In [ ]:
resposta = 0  # acumulando a resposta total
for i in range(pt): # somamos sobre todo o "passado"
    resposta = resposta +\
                tres_pulsos[i] * resp_impulso[pt - i] * passo 

plt.plot(t[pt], resposta, '*r')

É claro que não há nada de especial com o passo 15. Podemos calcular a resposta para todos os instantes de tempo, seguindo exatamente o mesmo procedimento da célula anterior: 

In [ ]:
plt.figure(9)
estimulo_resposta(t, resp_3p, tres_pulsos)  # figura dos itens acima

respostas = zeros(len(t))
for j in range(len(t)):
    for i in range(j): # somamos sobre todo o "passado"
        respostas[j] = respostas[j] +\
                tres_pulsos[i] * resp_impulso[j - i] * passo
plt.plot(t, respostas, 'o', fillstyle='none')

Essa operação que acabamos de realizar (nos laços `for` aninhados) chama-se **convolução**. Ela é amplamente utilizada e extremamente importante, e pode ser calculada diretamente (sem o duplo `for` que fizemos acima, por exemplo) em utilizando a função `convolve` do `numpy`. 

`convolve` calcula a convolução de duas listas de tamanho $m$ e $n$, retornando uma lista de tamanho $m+n-1$. Se a primeira lista é o estímulo e a segunda a resposta ao impulso unitário, seria natural esperar que a saída tivesse tamanho $m$. Entretanto ela é maior, para que consigamos ter a resposta completa, até os últimos pontos da entrada. Costumamos então de truncar o resultado da convolução ao tamanho da entrada:

In [ ]:
from numpy import convolve
plt.figure(10)
estimulo_resposta(t, resp_3p, tres_pulsos)  # figura do item 4

# convoluímos o estímulo com a resposta ao impulso
convolucao = convolve(tres_pulsos, resp_impulso) * passo
resposta = convolucao[:len(tres_pulsos)]  # truncamos 

plt.plot(t, resposta, 'o', fillstyle='none')

## Resposta a estímulos arbitrários

Finalmente, de posse da solução exata para a resposta ao impulso unitário e da operação de convolução, **é possível determinar a resposta de qualquer sistema linear a qualquer sinal de entrada**.

Utilizaremos a função `numpy.random.rand(n)`, que retorna `n` números aleatórios uniformemente distribuídos no intervalo $[0,1)$, para gerar um sinal de entrada. Calcularemos então a resposta da membrana a este sinal utilizando o método acima.

In [ ]:
from numpy.random import rand
from numpy import floor, round
plt.figure(11)
dt = 0.01
t = arange(400) * dt

entrada = floor(rand(len(t))+.05)*round(rand(len(t))*5)/dt
# convoluímos o estímulo com a resposta ao impulso
convolucao = convolve(entrada, resposta_impulso(t, 0.5))
resposta = convolucao[:len(entrada)]  * passo

estimulo_resposta(t, resposta, entrada)  # figura do item 4

## Cascatas de Integradores

É comum modelar sistemas sensoriais como uma "cascata" de integradores com vazamento, ou seja, com a saída de um integrador conectada à entrada do próximo. Na terminologia de redes neurais, isto equivale a uma rede "feedforward".


In [ ]:
# simulando cascata de integradores com vazamento
dt = 0.01
passos = 400
tau = 0.2
t = arange(passos) * dt
entrada = zeros(len(t)) 
entrada[0] = 1 / dt  # pulso unitário

n_cascata = 4  # 4 neurônios na cascata
saida = entrada
for i in range(n_cascata):
    saida = resposta_membrana(saida/tau, tau, passos, dt)[1] 
    
plt.figure(12)

estimulo_resposta(t, saida, entrada)

### Resposta ao impulso

Pode-se mostrar que a resposta obtida acima corresponde à [distribuição Gama](https://en.wikipedia.org/wiki/Gamma_distribution), com parâmetro de escala $(\theta)$ sendo a constante temporal $\tau$ de cada neurônio, e parâmetro de forma $(k)$ igual ao número de neurônios na cascata (inteiro). Com esta parametrização, temos:

$$
\mathrm{Gama}(t) = \left(\frac{t}{\theta}\right)^{k-1}\frac{e^{-\frac{t}{\theta}}}{\theta (k-1)!} \qquad k \in \mathbb{N}  
$$

In [ ]:
from math import factorial

def gama(t, theta, k):
    tt = t/theta
    return tt ** (k - 1) * exp(-tt) / theta / factorial(k-1) 

h = gama(t, tau, n_cascata) # resposta ao impulso da cascata
plt.plot(t, h)

### Resposta a estímulo ruidoso

Tendo a resposta ao impulso para a sequência de integradores, podemos calcular qual é a saída para uma entrada ruidosa. Neste caso, tomaremos números normalmente distribuídos, o que pode ser feito facilmente usando `numpy.random.randn(amostras)`:

In [ ]:
from numpy.random import randn

entrada = randn(len(t))/dt

saida = convolve(entrada, h)[:len(t)]

plt.figure(13)
estimulo_resposta(t, saida, entrada)

## Aplicação: Ressonância Magnética Funcional

Para uma aplicação deste tipo de modelo na modelagem de sinais de Ressonância Magnética Funcional (fMRI), veja:

Boynton, Geoffrey M., Stephen A. Engel, Gary H. Glover, and David J. Heeger. "Linear systems analysis of functional magnetic resonance imaging in human V1." _Journal of Neuroscience_ 16, no. 13 (1996): 4207-4221.